<a href="https://colab.research.google.com/github/orifelszer/CrimeData/blob/eden-branch/DNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
import zipfile
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import LabelEncoder
from sklearn.utils.class_weight import compute_class_weight

In [ ]:
# ✅ שלב 1: שכפול המאגר מה-GitHub
!git clone https://github.com/orifelszer/CrimeData.git

# ✅ שלב 2: פונקציה לפתיחת קובצי ZIP מהתיקייה המקומית
def load_zipped_csv_local(zip_path):
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        csv_name = zip_ref.namelist()[0]
        with zip_ref.open(csv_name) as file:
            return pd.read_csv(file)

# ✅ שלב 3: קריאת הנתונים לאחר השכפול (נתיב מקומי)
X_train = load_zipped_csv_local('CrimeData/X_train_supervised.zip')
X_test = load_zipped_csv_local('CrimeData/X_test_supervised.zip')
y_city_train = load_zipped_csv_local('CrimeData/y_city_train_supervised.zip')
y_city_test = load_zipped_csv_local('CrimeData/y_city_test_supervised.zip')
y_crime_train = load_zipped_csv_local('CrimeData/y_crime_train_supervised.zip')
y_crime_test = load_zipped_csv_local('CrimeData/y_crime_test_supervised.zip')

# ✅ בדיקה שהנתונים נטענו כראוי
print(f"X_train Shape: {X_train.shape}")
print(f"X_test Shape: {X_test.shape}")
print(f"y_city_train Shape: {y_city_train.shape}")
print(f"y_crime_train Shape: {y_crime_train.shape}")

In [ ]:
# ✅ פונקציה לבניית רשת נוירונים רב משימתית
def create_multitask_neural_network(input_shape, city_output_shape, crime_output_shape):
    inputs = keras.Input(shape=input_shape)

    # שכבות חבויות
    x = layers.Dense(256, activation='relu')(inputs)
    x = layers.Dropout(0.3)(x)
    x = layers.Dense(128, activation='relu')(x)
    x = layers.Dropout(0.3)(x)
    x = layers.Dense(64, activation='relu')(x)

    # יציאות נפרדות עבור העיר וסוג הפשע
    city_output = layers.Dense(city_output_shape, activation='softmax', name="city_output")(x)
    crime_output = layers.Dense(crime_output_shape, activation='softmax', name="crime_output")(x)

    # בניית המודל והגדרות אובדן ואופטימיזציה
    model = keras.Model(inputs=inputs, outputs=[city_output, crime_output])
    model.compile(optimizer='adam',
                  loss={'city_output': 'categorical_crossentropy',
                        'crime_output': 'categorical_crossentropy'},
                  metrics={'city_output': 'accuracy', 'crime_output': 'accuracy'})
    return model

In [ ]:
# ✅ יצירת הרשת
model = create_multitask_neural_network(input_shape=(X_train.shape[1],),
                                        city_output_shape=y_city_train.shape[1],
                                        crime_output_shape=y_crime_train.shape[1])

# ✅ סיכום מבנה הרשת
model.summary()

In [ ]:
def data_generator(X, y_city, y_crime, batch_size=64):
    num_samples = X.shape[0]
    while True:
        for offset in range(0, num_samples, batch_size):
            X_batch = X[offset:offset+batch_size].values.astype('float32')
            y_city_batch = y_city[offset:offset+batch_size].values.astype('float32')
            y_crime_batch = y_crime[offset:offset+batch_size].values.astype('float32')
            yield X_batch, {'city_output': y_city_batch, 'crime_output': y_crime_batch}

In [ ]:
# יצירת הגנרטורים
train_gen = data_generator(X_train, y_city_train, y_crime_train, batch_size=64)
val_gen = data_generator(X_test, y_city_test, y_crime_test, batch_size=64)

In [ ]:
# ✅ הגדרת Early Stopping
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True)

In [ ]:
# ✅ פונקציה להצגת גרפי אימון
def plot_training_history(history):
    plt.figure(figsize=(12, 5))

    # גרף דיוק
    plt.subplot(1, 2, 1)
    plt.plot(history.history['city_output_accuracy'], label='City Train Accuracy')
    plt.plot(history.history['val_city_output_accuracy'], label='City Test Accuracy')
    plt.plot(history.history['crime_output_accuracy'], label='Crime Train Accuracy')
    plt.plot(history.history['val_crime_output_accuracy'], label='Crime Test Accuracy')
    plt.title('Accuracy per Epoch')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()

    # גרף הפסד (Loss)
    plt.subplot(1, 2, 2)
    plt.plot(history.history['city_output_loss'], label='City Train Loss')
    plt.plot(history.history['val_city_output_loss'], label='City Test Loss')
    plt.plot(history.history['crime_output_loss'], label='Crime Train Loss')
    plt.plot(history.history['val_crime_output_loss'], label='Crime Test Loss')
    plt.title('Loss per Epoch')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()

    plt.tight_layout()
    plt.show()

In [ ]:
# ✅ אימון המודל עם הגנרטור
history = model.fit(
    train_gen,
    steps_per_epoch=len(X_train) // 64,
    validation_data=val_gen,
    validation_steps=len(X_test) // 64,
    epochs=50,
    callbacks=[early_stopping])

In [ ]:
# ✅ הצגת גרף האימון
plot_training_history(history)

In [ ]:
# ✅ הערכת ביצועי המודל על סט הבדיקה
results = model.evaluate(X_test_scaled,
                         {'city_output': y_city_test.values, 'crime_output': y_crime_test.values})
print(f"\nEvaluation Results: {results}")